In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sns
import geopandas as gpd

import cartopy.crs as ccrs
import scipy.stats as stats

from matplotlib import patches
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable

import sys
sys.path.append(os.path.abspath(".."))
from function import DOWN_raw
from function import ART_preprocessing as ART_pre

import warnings
warnings.filterwarnings('ignore')

## Validation using xarray to identify close point 

In [2]:
QC_method = 'QCv1_Flag1'

In [3]:
lon_min, lon_max, lat_min, lat_max, area, toll = 6.5, 19, 36.5, 48, 'ITALY', 0.002

Tr = [5,  10,  20,  50, 100, 200]
Fi = 1 - 1/np.array(Tr)

In [4]:
veneto_dir = os.path.join('/','media','arturo','T9','Data','shapes','Europa','Italy')

if os.path.exists(veneto_dir):
    REGIONS = gpd.read_file(os.path.join(veneto_dir,'Italy_regions.geojson'))
else:
    raise SystemExit(f"File not found: {veneto_dir}")

# update name of REGIONS geopandas
REGIONS = REGIONS.rename(columns={"reg_name": "Region"})
REGIONS['Region'] = REGIONS['Region'].replace('Emilia Romagna', 'Emilia-Romagna')

In [5]:
obs_base = os.path.join('/','media','arturo','T9','Data','Italy', 'stations', 'Italy_raingauges', 'QC_MIX')

In [6]:
METADATA = pd.read_csv(os.path.join(obs_base, 'METADATA', 'METADATA_INTENSE_UPDATE.csv'))
METADATA["Lat"] = np.round(METADATA["Lat"], 6)
METADATA["Lon"] = np.round(METADATA["Lon"], 6)
METADATA.head(2)

,Unnamed: 0,File,ISO,Region,Code,Name,Elevation,Or_EPSG,CRS_E,CRS_N,...,Source,TimeZon,Elevation.1,Orig_EPSG,Year_Start,Year_End,INTENSE_Resp,TimeZone,elevation_DEM,Elevation_update
0,0,IT-322_AA_6535_0060_QCv1.csv,IT-322,Alto-Adige,AA_6535,VANDOIESDISOPRA,746.0,32632.0,710057.0,5187849.0,...,Alto-Adige,UTC,NaN,NaN,NaN,NaN,NaN,NaN,745.9,746.0
1,1,IT-322_AA_5650_0060_QCv1.csv,IT-322,Alto-Adige,AA_5650,SELVADEIMOLINI,1141.0,32632.0,718972.0,5196967.0,...,Alto-Adige,UTC,NaN,NaN,NaN,NaN,NaN,NaN,1140.0,1141.0


In [7]:
ISO_names = pd.read_csv(os.path.join('/','media','arturo','T9','Data','Italy', 'stations', 'Italy_raingauges', 'ISO_IT_REGION_EN.csv'))
ISO_names["ISO_num"] = ISO_names["ISO"].str.split("-").str[1].astype(int)
ISO_names = ISO_names.sort_values("ISO_num").drop(columns="ISO_num").reset_index(drop=True)
ISO_names.head(5)

,Region,ISO
0,Piedmont,IT-210
1,Aosta Valley,IT-230
2,Lombardy,IT-250
3,Trentino,IT-321
4,Alto Adige,IT-322


In [8]:
Tr_index = 3
print(f'Tr: {Tr[Tr_index]} years')

Tr: 50 years


In [9]:
rr = 0

region_ISO = ISO_names.iloc[rr]['ISO']
region_label = ISO_names[ISO_names['ISO']==region_ISO]

print(f'{rr+1}: {region_label['Region'].values[0]} ({region_ISO})')

METADATA_clear = METADATA[METADATA['ISO']==region_ISO].reset_index(inplace=False)

SAT_WN, SAT_WC, SAT_WW = [], [], []
OBS_WN, OBS_WC, OBS_WW = [], [], []
LAT, LON, RE_50, ELEV = [], [], [], []
Qobs, Qest = [], []

for nn in range(len(METADATA_clear)): #len(METADATA_clear)
    filename = f'{METADATA_clear['File'].values[nn]}'
    lat_obs = METADATA_clear['Lat'][nn]
    lon_obs = METADATA_clear['Lon'][nn]
    elev_obs = METADATA_clear['Elevation_update'][nn]

    OBS_pd = pd.read_csv(os.path.join(obs_base, 'DATA_1dy', 'statistics', QC_method, region_ISO, filename))
    OBS_pd = OBS_pd[(OBS_pd['Year']>=2002)&(OBS_pd['Year']<=2023)].reset_index(drop=True)

    if len(OBS_pd) == 0:
        continue
    
    else:
        OBS_N = OBS_pd['N'].values
        OBS_C = OBS_pd['C'].values
        OBS_W = OBS_pd['W'].values
        OBS_Y = OBS_pd['Year'].values

        mask = ~np.isnan(OBS_N)

        OBS_N = OBS_N[mask]
        OBS_C = OBS_C[mask]
        OBS_W = OBS_W[mask]
        OBS_Y = OBS_Y[mask]

    if len(OBS_Y) >= 8: # greather than 8 years
        x0 = np.nanmean(OBS_C)
        OBS_Q, flag = DOWN_raw.mev_quant_update(Fi, x0, OBS_N, OBS_C, OBS_W, thresh=1)
        OBS_Q2 = np.where(flag, OBS_Q, np.nan)



1: Piedmont (IT-210)


In [13]:
OBS_pd

,Year,N,C,W
0,2002,85.0,13.523436,0.591103
1,2003,61.0,9.093676,0.834140
2,2004,80.0,13.128095,0.672631
3,2005,76.0,10.007097,0.727427
4,2006,81.0,8.377134,0.558224
5,2007,69.0,12.513448,0.773158
6,2008,126.0,11.913923,0.737650
7,2009,93.0,11.927963,0.654469
8,2010,112.0,10.578342,0.689240


In [14]:
OBS_Q

array([169.34775092, 213.15029913, 260.06492418, 327.82038586,
       383.77117556, 443.91376977])

In [ ]:
distances = ART_pre.haversine(lat2d_sat, lon2d_sat, lat_obs, lon_obs)

array([169.34775092, 213.15029913, 260.06492418, 327.82038586,
       383.77117556, 443.91376977])